### 10 Folium/Altair Maps for Geospatial Analytics in Minutes

Practical, copy-paste map recipes that turn raw lat/longs and polygons into decisions — without becoming a GIS expert.

In [1]:
# %pip install pandas numpy

In [1]:
# Setup & Tiny Dataset (once)

import pandas as pd
import numpy as np

# 1) Random points roughly around Bengaluru
rng = np.random.default_rng(42)
N = 1000
df_points = pd.DataFrame({
    "lat": 12.9 + rng.normal(0, 0.12, N),
    "lon": 77.6 + rng.normal(0, 0.12, N),
    "value": rng.integers(1, 100, N),
    "category": rng.choice(["A","B","C"], N, p=[0.5,0.3,0.2])
})

# 2) Minimal GeoJSON-like polygon (pretend districts; keep small)
districts = {
  "type":"FeatureCollection",
  "features":[
    {"type":"Feature","properties":{"name":"North"},"geometry":{"type":"Polygon","coordinates":[[[77.45,13.02],[77.9,13.02],[77.9,13.25],[77.45,13.25],[77.45,13.02]]]}},
    {"type":"Feature","properties":{"name":"South"},"geometry":{"type":"Polygon","coordinates":[[[77.45,12.55],[77.9,12.55],[77.9,12.9],[77.45,12.9],[77.45,12.55]]]}},
  ]
}

#### Folium: interactive analysis you can hand to anyone
1) Base map + smart marker clusters
When: Quick exploration across thousands of points without drowning in pins.



In [1]:
# %pip install folium

In [2]:
# %pip install folium

import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[12.97, 77.59], zoom_start=11, tiles="CartoDB positron")
cluster = MarkerCluster().add_to(m)

for _, r in df_points.sample(500).iterrows():
    folium.Marker(
        location=[r.lat, r.lon],
        popup=f"value: {r.value}, cat: {r.category}"
    ).add_to(cluster)

m


#### 2) Heatmap for density hot zones
When: You want where-are-people-concentrated without worrying about bins.

In [3]:
from folium.plugins import HeatMap

m = folium.Map(location=[12.97, 77.59], zoom_start=11, tiles="CartoDB dark_matter")
HeatMap(df_points[["lat","lon","value"]].values.tolist(), radius=18, blur=15).add_to(m)
m

### 3) Choropleth over simple polygons
When: Region-level rollups (districts, sales territories) tell the real story.

In [4]:
import json
gjson = json.loads(json.dumps(districts))

# fake regional metric
region_df = pd.DataFrame({"name":["North","South"], "score":[68, 45]})

m = folium.Map(location=[12.97,77.59], zoom_start=10, tiles="CartoDB positron")
folium.Choropleth(
    geo_data=gjson,
    data=region_df,
    columns=("name","score"),
    key_on="feature.properties.name",
    fill_color="YlOrRd",
    fill_opacity=0.75,
    line_opacity=0.6,
    legend_name="Score"
).add_to(m)
m

#### 4) Category-styled circle markers with layer control
When: Compare patterns by segment without making separate maps.

In [5]:
from folium import FeatureGroup, LayerControl

m = folium.Map(location=[12.97, 77.59], zoom_start=11)
colors = {"A": "#2ecc71", "B": "#e67e22", "C": "#3498db"}

for cat, group in df_points.groupby("category"):
    fg = FeatureGroup(name=f"Category {cat}", show=True).add_to(m)
    # Dynamically adjust sample size
    sample_size = min(250, len(group))
    for _, r in group.sample(sample_size).iterrows():
        folium.CircleMarker(
            [r.lat, r.lon], radius=5, color=colors[cat], fill=True, fill_opacity=0.6
        ).add_to(fg)

LayerControl(collapsed=False).add_to(m)
m

### 5) Time slider for evolving intensity
When: Show temporal drift without building a dashboard.

In [6]:
from folium.plugins import HeatMapWithTime

# simple time buckets
df_points["bucket"] = pd.cut(df_points["value"], bins=[0,33,66,100], labels=["low","mid","high"])
by_bucket = [df_points[df_points.bucket==b][["lat","lon","value"]].values.tolist()
             for b in ["low","mid","high"]]

m = folium.Map(location=[12.97,77.59], zoom_start=11)
HeatMapWithTime(by_bucket, index=["low","mid","high"], radius=14, auto_play=True).add_to(m)
m

#### 6) Scatter map with size & color encodings
When: Correlate two metrics spatially without the clutter of HTML widgets.

In [7]:
# %pip install altair
import altair as alt
alt.data_transformers.disable_max_rows()

base = alt.Chart(df_points).mark_circle(opacity=0.6).encode(
    longitude="lon:Q",
    latitude="lat:Q",
    size=alt.Size("value:Q", scale=alt.Scale(range=[10, 600])),
    color=alt.Color("category:N", scale=alt.Scale(scheme="tableau20")),
    tooltip=["lat","lon","value","category"]
).properties(width=800, height=450).project(type="mercator")
base

alt.Chart(...)

#### 7) Hexbin density for stable patterns
When: You need stable, grid-based density (no kernel bandwidth debates).

In [8]:
# approximate hex by transforming lon/lat to a grid
df = df_points.copy()
df["xbin"] = (df.lon * 100).round()
df["ybin"] = (df.lat * 100).round()

hexmap = alt.Chart(df).mark_rect().encode(
    x="xbin:Q", y="ybin:Q",
    color=alt.Color("count():Q", scale=alt.Scale(scheme="inferno")),
    tooltip=[alt.Tooltip("count():Q", title="count")]
).properties(width=800, height=450).project(type="mercator")

hexmap

alt.Chart(...)

#### 8) Choropleth with crisp borders
When: Region scores need print-ready clarity.

In [11]:
import altair as alt
import pandas as pd

regions = pd.json_normalize(districts["features"]).assign(
    name=lambda d: d["properties.name"],
    coords=lambda d: d["geometry.coordinates"]
)

# Convert polygons for Altair (one ring per feature)
def poly_to_rows(coords, name):
    # If coords is a single ring (list of [lon, lat] pairs)
    if isinstance(coords[0], list) and isinstance(coords[0][0], (float, int)):
        for lon, lat in coords:
            yield {"lon": lon, "lat": lat, "name": name}
    # If coords contains multiple rings (nested lists)
    elif isinstance(coords[0][0], list):
        for ring in coords:
            for lon, lat in ring:
                yield {"lon": lon, "lat": lat, "name": name}
    else:
        raise ValueError(f"Unexpected coordinates structure: {coords}")

poly_df = pd.DataFrame([r for _, row in regions.iterrows() for r in poly_to_rows(row.coords, row.name)])
metrics = pd.DataFrame({"name":["North","South"], "score":[68, 45]})

plot = alt.Chart(poly_df).mark_geoshape(stroke="#222", strokeWidth=1).encode(
    longitude="lon:Q", latitude="lat:Q",
    color=alt.Color("score:Q", scale=alt.Scale(scheme="orangered")),
).transform_lookup(
    lookup="name",
    from_=alt.LookupData(metrics, "name", ["score"])
).properties(width=800, height=450)

plot

alt.Chart(...)

### 9) Origin–destination lines (flow map)
When: Visualize flows without a full network stack.

In [12]:
# sample hubs
hubs = pd.DataFrame({
    "src_lat":[12.96,12.95,12.98],
    "src_lon":[77.55,77.65,77.70],
    "dst_lat":[12.90,13.00,12.85],
    "dst_lon":[77.70,77.58,77.62],
    "volume":[120, 40, 75]
})

flows = alt.Chart(hubs).mark_rule().encode(
    longitude="src_lon:Q", latitude="src_lat:Q",
    longitude2="dst_lon:Q", latitude2="dst_lat:Q",
    strokeWidth=alt.StrokeWidth("volume:Q", scale=alt.Scale(range=[1,8])),
    color=alt.value("#ff7f0e"),
    tooltip=["volume"]
).properties(width=800, height=450).project(type="mercator")
flows

alt.Chart(...)

### 10) Small-multiples for quick comparisons
When: Compare segments without interactive toggles.

In [13]:
facet = base.encode().facet(column="category:N").resolve_scale(color="independent", size="independent")
facet

alt.FacetChart(...)